# MLOps with Spark MLLib & Vertex AI Pipelines
In the prior lab modules - <br>(1) We authored Spark code in Serverless Spark interactive notebooks in Vertex AI workbench. <br>(2) We then created PySpark scripts off of them and tested them manually on cloud shell. <br> In this notebook -<br> (3) We will create a Vertex AI pipeline for MLOps - essentially chaining together the serverless Spark applications we developed in (2)<br> In a subsequent modules -<br>(4) We will create a Google Cloud Function to execute the pipeline and <br> (5) Finally, we will call that Google Cloud Function via Cloud Scheduler to complete the automation.
We will compile the pipeline JSON and use the same to schedule with Cloud Scheduler separately.

Dependency: Custom container image for Serverless Spark (already) created as part of (your) Terraform-based environment provisioning if you have created your environment using the Terraform provided in this lab.

  Copyright 2022 Google LLC
 
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at
 
       http://www.apache.org/licenses/LICENSE-2.0
 
  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License.

### 1. One time setup of dependencies
Uncomment the cell below, and run just the cell, one time ONLY to install necessary libraries - AND THEN comment it back.

In [ ]:
"""
!pip3 install --user --upgrade google-cloud-aiplatform==1.11.0 kfp==1.8.11 google-cloud-pipeline-components==1.0.1 --quiet --no-warn-conflicts

# Automatically restart kernel after installs
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
"""

### 2. Variables definition

In [ ]:
import random
from pathlib import Path as path
from typing import NamedTuple
import os

from google.cloud import aiplatform as vertex_ai
from google_cloud_pipeline_components import aiplatform as vertex_ai_components
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import (Artifact, ClassificationMetrics, Condition, Input,
                        Metrics, Output, component)

#### a. Project specifics

In [ ]:
import os

PROJECT_ID = ""
PROJECT_NBR = ""
UNIQUE_ID = random.randint(1, 10000)
WITHOUT_TASK_CACHING = True
BYO_NETWORK = True

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    project_id_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = project_id_output[0]
    print("Project ID: ", PROJECT_ID)
    
    
    project_nbr_output = !gcloud projects describe $PROJECT_ID --format='value(projectNumber)'
    PROJECT_NBR = project_nbr_output[0]
    print("Project Number: ", PROJECT_NBR)
    
umsa_output = !gcloud config list account --format "value(core.account)"
UMSA_FQN = umsa_output[0]
print("UMSA FQN: ", UMSA_FQN)
print("UNIQUE ID: ", UNIQUE_ID)

!gcloud config set project $PROJECT_ID

#### b. Local resources

In [ ]:
APP_BASE_NM = "customer-churn-model"

In [ ]:
LOCAL_SCRATCH_DIR = path(f"/home/jupyter/scratch/{APP_BASE_NM}/")

In [ ]:
!mkdir -m 777 -p $LOCAL_SCRATCH_DIR

In [ ]:
!ls -al $LOCAL_SCRATCH_DIR

In [ ]:
!cd $LOCAL_SCRATCH_DIR && pwd

In [ ]:
!ls -al /home/jupyter

#### d. The pre-created resources

In [ ]:
CODE_BUCKET = f"gs://USER_ID-s8s_code_bucket-{PROJECT_NBR}"
DATA_BUCKET = f"gs://USER_ID-s8s_data_bucket-{PROJECT_NBR}"
MODEL_BUCKET = f"gs://USER_ID-s8s_model_bucket-{PROJECT_NBR}"
SCRATCH_BUCKET = f"s8s-spark-bucket-{PROJECT_NBR}"
BQ_DS_NM = f"{PROJECT_ID}.USER_ID_customer_churn_ds"
LOCATION = "YOUR_GCP_LOCATION"
VPC_NM = f"s8s-vpc-{PROJECT_NBR}"
SUBNET_RESOURCE_URI = f"projects/{PROJECT_ID}/regions/{LOCATION}/subnetworks/spark-snet"
PERSISTENT_SPARK_HISTORY_SERVER_RESOURCE_URI = f"projects/{PROJECT_ID}/regions/{LOCATION}/clusters/s8s-sphs-{PROJECT_NBR}"
GCR_REPO_NM = f"s8s-spark-{PROJECT_NBR}"
DOCKER_IMAGE_TAG = "YOUR_SPARK_CONTAINER_IMAGE_TAG"
DOCKER_IMAGE_NM = "customer_churn_image"
DOCKER_IMAGE_FQN = f"gcr.io/{PROJECT_ID}/{DOCKER_IMAGE_NM}:{DOCKER_IMAGE_TAG}"

#### e. Pipeline entity specific

In [ ]:
PIPELINE_ID = UNIQUE_ID
PIPELINE_NM = f"USER_ID-{APP_BASE_NM}-pipeline"
PIPELINE_PACKAGE_SRC_LOCAL_PATH = f"{LOCAL_SCRATCH_DIR}/pipeline_{PIPELINE_ID}.json"
PIPELINE_ROOT_GCS_URI = f"{MODEL_BUCKET}/{APP_BASE_NM}/pipelines"

print('PIPELINE_ID =',PIPELINE_ID)
print('PIPELINE_NM =',PIPELINE_NM)
print('PIPELINE_PACKAGE_SRC_LOCAL_PATH =',PIPELINE_PACKAGE_SRC_LOCAL_PATH)
print('PIPELINE_ROOT_GCS_URI =',PIPELINE_ROOT_GCS_URI)

#### d. Pipeline stage agnostic

In [ ]:
PY_SCRIPTS_FQP = f"{CODE_BUCKET}/pyspark"
PYSPARK_COMMON_UTILS_SCRIPT_FQP = [f"{PY_SCRIPTS_FQP}/common_utils.py"]

print('PY_SCRIPTS_FQP =',PY_SCRIPTS_FQP)
print('PYSPARK_COMMON_UTILS_SCRIPT_FQP =',PYSPARK_COMMON_UTILS_SCRIPT_FQP)

#### e. Data preprocessing stage specific

In [ ]:
DATA_PREPROCESSING_BATCH_PREFIX = "preprocessing"
DATA_PREPROCESSING_BATCH_INSTANCE_ID = f"USER_ID-{APP_BASE_NM}-{DATA_PREPROCESSING_BATCH_PREFIX}-{UNIQUE_ID}"
DATA_PREPROCESSING_MAIN_PY_SCRIPT = f"{PY_SCRIPTS_FQP}/preprocessing.py"

DATA_PROCESSING_SINK = f"{BQ_DS_NM}.training_data"
DATA_PROCESSING_BQ_SINK_URI = f"bq://{DATA_PROCESSING_SINK}"

DATA_PREPROCESSING_ARGS = [f"--pipelineID={UNIQUE_ID}", \
        f"--projectID={PROJECT_ID}", \
        f"--projectNbr={PROJECT_NBR}", 
        f"--displayPrintStatements={True}"]

print('DATA_PREPROCESSING_BATCH_INSTANCE_ID =',DATA_PREPROCESSING_BATCH_INSTANCE_ID)
print('DATA_PREPROCESSING_MAIN_PY_SCRIPT =',DATA_PREPROCESSING_MAIN_PY_SCRIPT)
print('DATA_PROCESSING_SINK =',DATA_PROCESSING_SINK)
print('DATA_PROCESSING_BQ_SINK_URI =',DATA_PROCESSING_BQ_SINK_URI)
print('DATA_PREPROCESSING_ARGS =',DATA_PREPROCESSING_ARGS)

#### f. Dataset registration specific

In [ ]:
MANAGED_DATASET_NM = f"USER_ID-{APP_BASE_NM}-{UNIQUE_ID}"

#### g. Model specific

In [ ]:
MODEL_TRAINING_BATCH_PREFIX = "training"
MODEL_TRAINING_BATCH_INSTANCE_ID = f"USER_ID-{APP_BASE_NM}-{MODEL_TRAINING_BATCH_PREFIX}-{UNIQUE_ID}"
MODEL_TRAINING_MAIN_PY_SCRIPT = f"{PY_SCRIPTS_FQP}/model_training.py"
MODEL_TRAINING_ARGS = [f"--pipelineID={UNIQUE_ID}", \
        f"--projectID={PROJECT_ID}", \
        f"--projectNbr={PROJECT_NBR}", 
        f"--displayPrintStatements={True}"]

MODEL_METRICS_BUCKET_FQP = f"gs://USER_ID-s8s_metrics_bucket-{PROJECT_NBR}/{APP_BASE_NM}/{MODEL_TRAINING_BATCH_PREFIX}/{UNIQUE_ID}/full/metrics.json"

print('MODEL_TRAINING_BATCH_INSTANCE_ID =',MODEL_TRAINING_BATCH_INSTANCE_ID)
print('MODEL_TRAINING_MAIN_PY_SCRIPT =',MODEL_TRAINING_MAIN_PY_SCRIPT)
print('MODEL_TRAINING_ARGS =',MODEL_TRAINING_ARGS)
print('MODEL_METRICS_BUCKET_FQP =',MODEL_METRICS_BUCKET_FQP)

#### h. Hyperparameter tuning specific

In [ ]:
# Condition
AUPR_THRESHOLD = 0.5
AUPR_HYPERTUNE_CONDITION = "[AUPR_HYPERTUNE]"

HYPERPARAMETER_TUNING_BATCH_PREFIX = "hyperparameter-tuning"
HYPERPARAMETER_TUNING_BATCH_INSTANCE_ID = f"USER_ID-{APP_BASE_NM}-{HYPERPARAMETER_TUNING_BATCH_PREFIX}-{UNIQUE_ID}"
HYPERPARAMETER_TUNING_ARGS = [f"--pipelineID={UNIQUE_ID}", \
        f"--projectID={PROJECT_ID}", \
        f"--projectNbr={PROJECT_NBR}", 
        f"--displayPrintStatements={True}"]

HYPERPARAMETER_TUNING_MAIN_PY_SCRIPT = f"{PY_SCRIPTS_FQP}/hyperparameter_tuning.py"
HYPERPARAMETER_TUNING_BUCKET_FQP = f"gs://USER_ID-s8s_metrics_bucket-{PROJECT_NBR}/{APP_BASE_NM}/{HYPERPARAMETER_TUNING_BATCH_PREFIX}/{UNIQUE_ID}/full/metrics.json"


print('HYPERPARAMETER_TUNING_BATCH_INSTANCE_ID =',HYPERPARAMETER_TUNING_BATCH_INSTANCE_ID)
print('HYPERPARAMETER_TUNING_MAIN_PY_SCRIPT =',HYPERPARAMETER_TUNING_MAIN_PY_SCRIPT)
print('HYPERPARAMETER_TUNING_ARGS =',HYPERPARAMETER_TUNING_ARGS)

### 3. Initialize Vertex AI SDK for Python

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=SCRATCH_BUCKET)

### 4. Define custom components

In [ ]:
@component(
    base_image="python:3.8",
    packages_to_install=["numpy==1.21.2", "pandas==1.3.3", "scikit-learn==0.24.2"],
)
def fnEvaluateModel(
    metricsUri: str,
    metrics: Output[Metrics],
    plots: Output[ClassificationMetrics],
) -> NamedTuple("Outputs", [("threshold_metric", float)]):

    import json
    import numpy as np
    from sklearn.metrics import confusion_matrix, roc_curve

    # Variables
    metricsGCSMountPath = metricsUri.replace("gs://", "/gcs/")
    labels = ["yes", "no"]

    # Helpers
    def fnCalculateROC(metrics, true, score):
        y_true_np = np.array(metrics[true])
        y_score_np = np.array(metrics[score])
        fpr, tpr, thresholds = roc_curve(
            y_true=y_true_np, y_score=y_score_np, pos_label=True
        )
        return fpr, tpr, thresholds

    def fnCalculateConfusionMatrix(metrics, true, prediction):
        y_true_np = np.array(metrics[true])
        y_pred_np = np.array(metrics[prediction])
        c_matrix = confusion_matrix(y_true_np, y_pred_np)
        return c_matrix

    # Main
    with open(metricsGCSMountPath, mode="r") as json_file:
        metricsDictionary = json.load(json_file)

    area_roc = metricsDictionary["test_area_roc"]
    area_prc = metricsDictionary["test_area_prc"]
    acc = metricsDictionary["test_accuracy"]
    f1 = metricsDictionary["test_f1"]
    prec = metricsDictionary["test_precision"]
    rec = metricsDictionary["test_recall"]

    metrics.log_metric("Test_areaUnderROC", area_roc)
    metrics.log_metric("Test_areaUnderPRC", area_prc)
    metrics.log_metric("Test_Accuracy", acc)
    metrics.log_metric("Test_f1-score", f1)
    metrics.log_metric("Test_Precision", prec)
    metrics.log_metric("Test_Recall", rec)

    fpr, tpr, thresholds = fnCalculateROC(metricsDictionary, "true", "score")
    c_matrix = fnCalculateConfusionMatrix(metricsDictionary, "true", "prediction")
    plots.log_roc_curve(fpr.tolist(), tpr.tolist(), thresholds.tolist())
    plots.log_confusion_matrix(labels, c_matrix.tolist())

    componentOutputsTuple = NamedTuple(
        "Outputs",
        [
            ("threshold_metric", float),
        ],
    )
    return componentOutputsTuple(area_prc)


### 5. Define Vertex AI Pipeline

##### Option 1: In this version we dont disable task level caching 

In [ ]:
@dsl.pipeline(
    name=PIPELINE_NM, 
    description="A SparkMLlib MLOps Vertex pipeline")
def fnSparkMlopsPipeline(
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    service_account: str = UMSA_FQN,
    subnetwork_uri: str = SUBNET_RESOURCE_URI,
    spark_phs_nm: str = PERSISTENT_SPARK_HISTORY_SERVER_RESOURCE_URI,
    container_image: str = DOCKER_IMAGE_FQN,
    common_utils_py_fqn: list = PYSPARK_COMMON_UTILS_SCRIPT_FQP,
    data_preprocessing_pyspark_batch_id: str = DATA_PREPROCESSING_BATCH_INSTANCE_ID,
    data_preprocessing_main_py_fqn: str = DATA_PREPROCESSING_MAIN_PY_SCRIPT,
    data_preprocessing_args: list = DATA_PREPROCESSING_ARGS,
    managed_dataset_display_nm: str = MANAGED_DATASET_NM,
    managed_dataset_src_uri: str = DATA_PROCESSING_BQ_SINK_URI,
    model_training_pyspark_batch_id: str = MODEL_TRAINING_BATCH_INSTANCE_ID,
    model_training_main_py_fqn: str = MODEL_TRAINING_MAIN_PY_SCRIPT,
    model_training_metrics_fqp: str = MODEL_METRICS_BUCKET_FQP,
    model_training_args: list = MODEL_TRAINING_ARGS,
    threshold: float = AUPR_THRESHOLD,
    hyperparameter_tuning_pyspark_batch_id: str = HYPERPARAMETER_TUNING_BATCH_INSTANCE_ID,
    hyperparameter_tuning_main_py_fqn: str = HYPERPARAMETER_TUNING_MAIN_PY_SCRIPT,
    hyperparameter_tuning_args: list = HYPERPARAMETER_TUNING_ARGS,
    hyperparameter_tuning_metrics_fqp: str = MODEL_METRICS_BUCKET_FQP,
):
    from google_cloud_pipeline_components.experimental.dataproc import \
        DataprocPySparkBatchOp

    # Step 1. PRE-PROCESS DATA in PREP FOR MODEL TRAINING
    # ....................................................................
    preprocessingStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = data_preprocessing_pyspark_batch_id,
        main_python_file_uri = data_preprocessing_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = data_preprocessing_args
    ).set_display_name("Preprocessing")
    
    
    # Step 2. REGISTER PRE-PROCESSED DATA AS MANAGED DATASET
    # ....................................................................
    createManagedDatasetStep = vertex_ai_components.TabularDatasetCreateOp(
        display_name= managed_dataset_display_nm,
        bq_source=managed_dataset_src_uri,
        project=project_id,
        location=location,
    ).after(preprocessingStep).set_display_name("Dataset registration")
    
    # Step 3. TRAIN MODEL
    # .................................................................... 
    trainSparkMLModelStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = model_training_pyspark_batch_id,
        main_python_file_uri = model_training_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = model_training_args
    ).after(preprocessingStep).set_display_name("Model training")
    
    # Step 4. EVALUATE MODEL
    # .................................................................... 
    evaluateModelStep = fnEvaluateModel(model_training_metrics_fqp).after(trainSparkMLModelStep).set_display_name("Evaluate model")
    
    # Step 5. CONDITIONAL HYPERPARAMETER TUNING
    # .................................................................... 
    with Condition(
        evaluateModelStep.outputs["threshold_metric"] >= threshold,
        name="AUPR Threshold Exceeded",
    ):
        # HYPERPARAMETER TUNING
        hyperparameterTuningStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = hyperparameter_tuning_pyspark_batch_id,
        main_python_file_uri = hyperparameter_tuning_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = hyperparameter_tuning_args
        ).after(evaluateModelStep).set_display_name("Hyperparameter tuning")

##### Option 2: In this version we disable task level caching
Prefer this for scheduled execution

In [ ]:
@dsl.pipeline(
    name=PIPELINE_NM, 
    description="A SparkMLlib MLOps Vertex pipeline")
def fnSparkMlopsPipelineWithoutCaching(
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    service_account: str = UMSA_FQN,
    subnetwork_uri: str = SUBNET_RESOURCE_URI,
    spark_phs_nm: str = PERSISTENT_SPARK_HISTORY_SERVER_RESOURCE_URI,
    container_image: str = DOCKER_IMAGE_FQN,
    common_utils_py_fqn: list = PYSPARK_COMMON_UTILS_SCRIPT_FQP,
    data_preprocessing_pyspark_batch_id: str = DATA_PREPROCESSING_BATCH_INSTANCE_ID,
    data_preprocessing_main_py_fqn: str = DATA_PREPROCESSING_MAIN_PY_SCRIPT,
    data_preprocessing_args: list = DATA_PREPROCESSING_ARGS,
    managed_dataset_display_nm: str = MANAGED_DATASET_NM,
    managed_dataset_src_uri: str = DATA_PROCESSING_BQ_SINK_URI,
    model_training_pyspark_batch_id: str = MODEL_TRAINING_BATCH_INSTANCE_ID,
    model_training_main_py_fqn: str = MODEL_TRAINING_MAIN_PY_SCRIPT,
    model_training_metrics_fqp: str = MODEL_METRICS_BUCKET_FQP,
    model_training_args: list = MODEL_TRAINING_ARGS,
    threshold: float = AUPR_THRESHOLD,
    hyperparameter_tuning_pyspark_batch_id: str = HYPERPARAMETER_TUNING_BATCH_INSTANCE_ID,
    hyperparameter_tuning_main_py_fqn: str = HYPERPARAMETER_TUNING_MAIN_PY_SCRIPT,
    hyperparameter_tuning_args: list = HYPERPARAMETER_TUNING_ARGS,
    hyperparameter_tuning_metrics_fqp: str = MODEL_METRICS_BUCKET_FQP,
):
    from google_cloud_pipeline_components.experimental.dataproc import \
        DataprocPySparkBatchOp

    # Step 1. PRE-PROCESS DATA in PREP FOR MODEL TRAINING
    # ....................................................................
    preprocessingStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = data_preprocessing_pyspark_batch_id,
        main_python_file_uri = data_preprocessing_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = data_preprocessing_args
    ).set_caching_options(False).set_display_name("Preprocessing")
    
    
    # Step 2. REGISTER PRE-PROCESSED DATA AS MANAGED DATASET
    # ....................................................................
    createManagedDatasetStep = vertex_ai_components.TabularDatasetCreateOp(
        display_name= managed_dataset_display_nm,
        bq_source=managed_dataset_src_uri,
        project=project_id,
        location=location,
    ).after(preprocessingStep).set_caching_options(False).set_display_name("Dataset registration")
    
    # Step 3. TRAIN MODEL
    # .................................................................... 
    trainSparkMLModelStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = model_training_pyspark_batch_id,
        main_python_file_uri = model_training_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = model_training_args
    ).set_caching_options(False).after(preprocessingStep).set_display_name("Model training")
    
    # Step 4. EVALUATE MODEL
    # .................................................................... 
    evaluateModelStep = fnEvaluateModel(model_training_metrics_fqp).after(trainSparkMLModelStep).set_caching_options(False).set_display_name("Evaluate model")
    
    # Step 5. CONDITIONAL HYPERPARAMETER TUNING
    # .................................................................... 
    with Condition(
        evaluateModelStep.outputs["threshold_metric"] >= threshold,
        name="AUPR Threshold Exceeded",
    ):
        # HYPERPARAMETER TUNING
        hyperparameterTuningStep = DataprocPySparkBatchOp(
        project = project_id,
        location = location,
        container_image = container_image,
        subnetwork_uri = subnetwork_uri,
        spark_history_dataproc_cluster = spark_phs_nm,
        service_account = service_account,     
        batch_id = hyperparameter_tuning_pyspark_batch_id,
        main_python_file_uri = hyperparameter_tuning_main_py_fqn,
        python_file_uris = common_utils_py_fqn,
        args = hyperparameter_tuning_args
        ).after(evaluateModelStep).set_caching_options(False).set_display_name("Hyperparameter tuning")


### 5. Compile the Vertex AI Pipeline into a JSON

In [ ]:
if WITHOUT_TASK_CACHING:
    compiler.Compiler().compile(pipeline_func=fnSparkMlopsPipelineWithoutCaching, package_path=PIPELINE_PACKAGE_SRC_LOCAL_PATH)
    print("Executing fnSparkMlopsPipelineWithoutCaching")
else:
    compiler.Compiler().compile(pipeline_func=fnSparkMlopsPipeline, package_path=PIPELINE_PACKAGE_SRC_LOCAL_PATH)
    print("Executing fnSparkMlopsPipeline")

### 6. Create the pipeline

In [ ]:
pipeline = vertex_ai.PipelineJob(
    display_name=PIPELINE_NM,
    template_path=PIPELINE_PACKAGE_SRC_LOCAL_PATH,
    pipeline_root=PIPELINE_ROOT_GCS_URI,
    enable_caching=False
)

### 7. Submit the Pipeline for execution
There are two options below, one that uses the customer specified network that is peered with Vertex AI tenant network, and one that uses Vertex AI tenant network altogether

In [ ]:
if BYO_NETWORK:
    pipeline.submit(service_account=UMSA_FQN, network=f"projects/{PROJECT_NBR}/global/networks/{VPC_NM}")
else:
    pipeline.submit(service_account=UMSA_FQN)

### 8. How do we automate this?

To automate this, we will take the compiled JSON, (1) test run it via Vertex AI Cloud Console UI, and then (2) schedule with Cloud Scheduler (calls Cloud Function that calls the Vertex AI REST endpoint for the Vertex AI pipeline).<br>This is a lab module that you can proceed to next.<br>
Note: The json below has your project details. The lab author has a de-identified version with placeholders for your information in the git repo ../04-templates/\*.json and as part of the Terraform automation, a customized version except custom pipeline ID is placed in 05-pipelines in your local directory in cloud shell. A copy of it is placed in GCS -> s8s-pipelines-bucket-YOUR_PROJECT_NUMBER/templates. At scheduled run time, a new custom pipeline ID is generated in the cloud function, and substituted in the json and placed in the bucket - s8s-pipelines-bucket-YOUR_PROJECT_NUMBER/execution and this is used to launch the pipeline. 

In [ ]:
! cat $LOCAL_SCRATCH_DIR/*